In [2]:
import pandas as pd
import numpy as np
from pct.tree.heuristic.Heuristic import Heuristic5
from pct.tree.heuristic.NumericHeuristic import NumericHeuristic5
from pct.tree.splitter.splitter import Splitter
from pct.tree.tree import Tree
# from pct.tree.ftest.ftest import FTest


## From YXH's data process

In [2]:

from sklearn.preprocessing import LabelEncoder

# Load the u.data dataset
u_data = pd.read_csv('/Users/camillecu/Downloads/KUL/master_thesis/master_thesis_github/ml-100k/u.data', sep='\t', header=None, names=['user_id', 'item_id', 'rating', 'timestamp'])

# # # Perform label encoding on user_id and item_id
# user_encoder = LabelEncoder()
# item_encoder = LabelEncoder()

# u_data['user_id'] = user_encoder.fit_transform(u_data['user_id'])
# u_data['item_id'] = item_encoder.fit_transform(u_data['item_id'])

# # # Create the rating matrix
# n_users = u_data['user_id'].nunique()
# n_items = u_data['item_id'].nunique()

# Define the threshold
THRESHOLD = 3  # Ratings above this are "Lovers", below are "Haters"

# Pivot the data
rating_matrix = u_data.pivot(index='user_id', columns='item_id', values='rating').fillna(0)
rating_matrix.index = u_data['user_id'].unique()  # Ensure user_id alignment
rating_matrix.columns = u_data['item_id'].unique()  # Ensure item_id alignment


# Apply threshold classification to the rating matrix (after thresholding)
rating_matrix_thresholded = np.where(rating_matrix > THRESHOLD, 1, np.where(rating_matrix > 0, -1, 0))

# Convert numpy array to pandas DataFrame for easier inspection
rating_matrix_thresholded_df = pd.DataFrame(rating_matrix_thresholded)


In [3]:

from sklearn.preprocessing import LabelEncoder

# Load the u.data dataset
u_data = pd.read_csv("/Users/camillecu/Downloads/KUL/master_thesis/master_thesis_github/ml-1m/ratings.txt",sep='::', header=None, names=['user_id', 'item_id', 'rating', 'timestamp'])


# Pivot the data
rating_matrix = u_data.pivot(index='user_id', columns='item_id', values='rating').fillna(0)
rating_matrix.index = u_data['user_id'].unique()  # Ensure user_id alignment
rating_matrix.columns = u_data['item_id'].unique()  # Ensure item_id alignment



/var/folders/cv/bbbq9ykj0jg6pt6c7tx5_vjh0000gn/T/ipykernel_74556/1450568430.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  u_data = pd.read_csv("/Users/camillecu/Downloads/KUL/master_thesis/master_thesis_github/ml-1m/ratings.txt",sep='::', header=None, names=['user_id', 'item_id', 'rating', 'timestamp'])


In [11]:
u_data

,user_id,item_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [12]:
rating_matrix

,1193,661,914,3408,2355,1197,1287,2804,594,919,...,127,3382,1843,286,3530,2198,2703,2845,3607,2909
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6039,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## initiate tree and splitter

In [ ]:
tree = Tree(min_instances=7)
tree.fit(rating_matrix, u_data['rating'], target_weights=None)




In [6]:
# Specify the level you want to retrieve nodes from
level = 2  # Example: Get nodes at level 2

# Call get_nodes_by_level and print the result
nodes_at_level = tree.get_nodes_by_level(level)

print(f"Nodes at level {level}:")
for node in nodes_at_level:
    print(f"Node ID: {node.id}")
    print(f"Item ID: {node.item_id}")
    print(f"User IDs: {node.user_ids}")
    print(f"Depth: {node.depth}")
    print("-" * 20)


Nodes at level 2:
Node ID: 40
Item ID: 202
User IDs: [1, 9, 13, 14, 526, 535, 540, 544, 55, 71, 72, 588, 599, 90, 610, 101, 620, 129, 644, 134, 650, 147, 151, 169, 172, 175, 183, 214, 246, 269, 795, 802, 808, 328, 329, 853, 857, 348, 357, 876, 882, 393, 908, 398, 405, 409, 420, 452, 456, 462, 499, 501, 511]
Depth: 2
--------------------
Node ID: 40
Item ID: 260
User IDs: [530, 536, 554, 57, 586, 604, 626, 123, 636, 124, 128, 642, 656, 661, 158, 162, 676, 692, 196, 709, 200, 725, 216, 728, 740, 241, 263, 359, 890, 894, 904, 927, 418, 434, 457, 460, 486, 505]
Depth: 2
--------------------
Node ID: 40
Item ID: 153
User IDs: [2, 3, 8, 24, 25, 30, 37, 41, 42, 56, 58, 59, 70, 74, 77, 78, 79, 83, 93, 95, 102, 104, 112, 115, 130, 137, 138, 166, 168, 176, 190, 201, 205, 236, 242, 243, 247, 250, 271, 282, 287, 289, 292, 321, 323, 324, 326, 336, 340, 342, 361, 385, 396, 397, 407, 436, 442, 444, 447, 470, 474, 478, 483, 488, 493, 498, 500, 507, 514, 518, 522, 534, 537, 546, 552, 553, 556, 573, 580

In [7]:
tree.print_tree_structure()

Leaf Node: No
Depth: 0
Item_ID: 471
user_ids: [196, 186, 22, 244, 166, 298, 115, 253, 305, 6, 62, 286, 200, 210, 224, 303, 122, 194, 291, 234, 119, 167, 299, 308, 95, 38, 102, 63, 160, 50, 301, 225, 290, 97, 157, 181, 278, 276, 7, 10, 284, 201, 287, 246, 242, 249, 99, 178, 251, 81, 260, 25, 59, 72, 87, 42, 292, 20, 13, 138, 60, 57, 223, 189, 243, 92, 241, 254, 293, 127, 222, 267, 11, 8, 162, 279, 145, 28, 135, 32, 90, 216, 250, 271, 265, 198, 168, 110, 58, 237, 94, 128, 44, 264, 41, 82, 262, 174, 43, 84, 269, 259, 85, 213, 121, 49, 155, 68, 172, 19, 268, 5, 80, 66, 18, 26, 130, 256, 1, 56, 15, 207, 232, 52, 161, 148, 125, 83, 272, 151, 54, 16, 91, 294, 229, 36, 70, 14, 295, 233, 214, 192, 100, 307, 297, 193, 113, 275, 219, 218, 123, 158, 302, 23, 296, 33, 154, 77, 270, 187, 170, 101, 184, 112, 133, 215, 69, 104, 240, 144, 191, 61, 142, 177, 203, 21, 197, 134, 180, 236, 263, 109, 64, 114, 239, 117, 65, 137, 257, 111, 285, 96, 116, 73, 221, 235, 164, 281, 182, 129, 45, 131, 230, 126, 231